In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from tqdm import *

In [2]:
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (15,15), 0)
    return processed_img

#Function to find features in the two imagess
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)
    
    #Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.2*n.distance:
            good.append([m])
    
    left_pts  = np.float32([kp1[m[0].queryIdx].pt for m in good])
    right_pts  = np.float32([kp2[m[0].trainIdx].pt for m in good])
    return left_pts, right_pts


In [3]:
def fundamental_matrix(left_pts, right_pts):
    A = []
    
    for i in range(8):
        src_x, src_y  = left_pts[i]
        dst_x, dst_y = right_pts[i]
        A_row = np.asarray([dst_x*src_x, dst_x*src_y, dst_x, dst_y*src_x, dst_y*src_y, dst_y, src_x, src_y, 1])
        A.append(A_row)
        
    A = np.asarray(A)
    U, S, V = np.linalg.svd(A)
    F = np.reshape(V[-1, :], (3, 3))
    u, s, vt = np.linalg.svd(F)
    s = np.diag(s)
    s[2, 2] = 0
    F = u @ (s @ vt)
    F = F / F[-1,-1]
    F = (abs(F) > (10**(-3))) * F
    return F

#Ransac function to find the best Fundamental Matrix that satisfies the epipolar constraint
def ransac(src_pts, dst_pts, iterations, threshold):
    number_of_points = len(src_pts)
    max_inlier_count = 0
    
    for i in range(iterations):
        inlier = 0
        random_indices = np.random.choice(number_of_points, size = 8, replace=False)
        random_src_pts = src_pts[random_indices, :]
        random_dst_pts = dst_pts[random_indices, :]
        
        F = fundamental_matrix(random_src_pts, random_dst_pts)

        for i in range(number_of_points):
            a, b = dst_pts[i], src_pts[i]
            a, b = np.append(a, 1).reshape((3, 1)), np.append(b, 1).reshape((3, 1))
            c = (a.T @ F) @ b
            
            if abs(c) <= threshold:
                inlier+=1
            
        if inlier > max_inlier_count:
            max_inlier_count = inlier
            best_F = F          
    return best_F

In [4]:
def essential_matrix(F, K1, K2):
    E = (K2.T @ F) @ K1
    U, _, V = np.linalg.svd(E)
    S = [1, 1, 0]
    S = np.diag(S)
    E = np.matmul(np.matmul(U, S), V)
    return E

In [5]:
def decompose_essential_matrix(E):
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    U, S, V = np.linalg.svd(E)
    
    R_set = []
    C_set = []
    
    R_set.append((U @ W) @ V)
    R_set.append((U @ W) @ V)
    R_set.append((U @ W.T) @ V)
    R_set.append((U @ W.T) @ V)
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])
    C_set.append(U[:, 2])
    C_set.append(-U[:, 2])

    for i in range(4):
        if (np.linalg.det(R_set[i]) < 0):
            R_set[i] = -R_set[i]
            C_set[i] = -C_set[i]

    return R_set, C_set

In [6]:
def projection_matrix(K, R, C):
    I  = np.identity(3)
    C = np.reshape(C, (3, 1))
    return (K @ (R @ np.hstack((I, -C))))

#Function to find 3D point given 2D pixel locations
def linear_triangulation(R_set, C_set, left_pts, right_pts, K1, K2):
    x3D_set = []

    for i in range(len(R_set)):
        R1, R2 = np.identity(3), R_set[i]
        C1, C2 = np.zeros((3, 1)),  C_set[i].reshape(3,1)
        
        P1 = projection_matrix(K1, R1, C1)
        P2 = projection_matrix(K2, R2, C2)
        
        p1, p2, p3 = P1
        p1_, p2_, p3_ = P2
        
        p1, p2, p3 =  p1.reshape(1,-1), p2.reshape(1,-1), p3.reshape(1,-1) 
        p1_, p2_, p3_ =  p1_.reshape(1,-1), p2_.reshape(1,-1), p3_.reshape(1,-1) 
        
        x3D =[]
        
        for left_pt, right_pt in zip(left_pts, right_pts):
            x, y = left_pt
            x_, y_ = right_pt
            A = np.vstack((y * p3 - p2, p1 - x * p3, y_ * p3_ - p2_, p1_-x_ * p3_ ))
            _, _, Vt = np.linalg.svd(A)
            X = Vt[-1]
            X = X / X[-1]
            x3D.append(X[:3])
         
        x3D_set.append(x3D)
    return x3D_set


In [7]:
def disambiguate_camera_pose(R_set, C_set, x3D_set):
    best_i = 0
    max_positive_depths = 0
    
    for i in range(len(R_set)):
        n_positive_depths=  0
        R, C = R_set[i],  C_set[i].reshape(-1, 1) 
        r3 = R[2].reshape(1, -1)
        x3D = x3D_set[i]
        
        for X in x3D:
            X = X.reshape(-1, 1) 
            if r3 @ (X - C) > 0 and X[2] > 0: 
                n_positive_depths += 1
                
        if n_positive_depths > max_positive_depths:
            best_i = i
            max_positive_depths = n_positive_depths 

    R, C = R_set[best_i], C_set[best_i]
    return R, C

#Function to find start and end co-ordinates given equation of an epiline
def epiline_coordinates(lines, img):
    lines = lines.reshape(-1, 3)
    c = img.shape[1]
    co_ordinates = []
    
    for line in lines:
        x0, y0 = map(int, [0, -line[2] / line[1]])
        x1, y1 = map(int, [c, -(line[2] + line[0] * c) / line[1]])
        co_ordinates.append([[x0, y0], [x1, y1]])
        
    return co_ordinates